## Understanding Natural Language Processing

NLP has the goal of derived information out of natural language (could be sequences text or speech). Another common term for NLP problems is sequence to sequence problems (seq2seq)

## Helper Functions

In [2]:
from helper_functions import unzip_data, plot_loss_curves, create_tensorboard_callback, compare_historys

## Visualizing the Text Dataset

In [3]:
import pandas as pd
train_df = pd.read_csv("nlp_getting_started/train.csv")
test_df = pd.read_csv("nlp_getting_started/test.csv")

In [4]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
# getting one data sample
train_df["text"][6845]

'in response to trauma Children of Addicts develop a defensive self - one that decreases vulnerability. (3'

In [6]:
# Shuffling the data so that the supervised model won't learn any patterns to follow the prediction
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [7]:
# Number of examples on the target class
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [8]:
len(train_df), len(test_df)

(7613, 3263)

In [9]:
# Visualize the random samples
import random
random_index = random.randint(0, len(train_df)-5) # less the length so the rand won't exceed it
for value in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  # itertuples are used to iterate over the dataframe rows and return them as tuples.
  _, text, target = value
  print(f"Target Value: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
  print(f"Text Value:\n{text}\n")
  print("---\n")

Target Value: 0 (not real disaster)
Text Value:
@ArianaGrande I literally walked out of the concert and screamed MY SOUL HAS BEEN BLESSED

---

Target Value: 0 (not real disaster)
Text Value:
@RosemaryTravale Do we all use the same weapon? 'cause we might be screwed XD

---

Target Value: 0 (not real disaster)
Text Value:
Stretcher brought out for Vampiro. Cut to commercial isn't a good sign. #UltimaLucha #LuchaUnderground

---

Target Value: 1 (real disaster)
Text Value:
Young children among those rescued from capsized boat off Libya http://t.co/Kot9zVD2H7 via @IrishTimesWorld

---

Target Value: 1 (real disaster)
Text Value:
Injuries Illnesses and Fatalities Latest Numbers : http://t.co/1uo1aTrbbJ

---



## Spliting Data into Train and Test Data

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled['text'].to_numpy(),
                                                                              train_df_shuffled['target'].to_numpy(),
                                                                              test_size=0.1, # 10% of the train data to be test data
                                                                              random_state=42)

In [13]:
len(train_sentences), len(val_sentences), len(train_labels), len(val_labels)

(6851, 762, 6851, 762)

## Converting Text into Numbers

Before building the model need to convert the text into numbers using:

1. Tokenization - directly map a token to a number
2. Embedding - create a matrix of feature vector for each token(the size of the feature vector can be defined and this embedding can be learned)

### Text Vectorization - Tokenization

In [15]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

In [16]:
# Use the default TextVectorization parameters
text_vectorizer = TextVectorization(max_tokens=None, # how many words in the vocabulary (automatically add <oov>)
                                    standardize='lower_and_strip_punctuation',
                                    split='whitespace',
                                    ngrams=None, # create groups of n-words
                                    output_mode="int", # how to map tokens to numbers
                                    output_sequence_length=None, # how long do you want your sequences to be
                                    #pad_to_max_tokens=True # pad the feature axis which are less the max token axis, can be used only when max_tokens is mentioned
)

In [17]:
train_sentences[0].split()

['@mogacola', '@zamtriossu', 'i', 'screamed', 'after', 'hitting', 'tweet']

In [18]:
len(train_sentences[0].split())

7

In [19]:
# Find the average number of tokens (words) in the training tweets
round(sum([len(i.split()) for i in train_sentences]))

102087

In [20]:
round(sum([len(i.split()) for i in train_sentences]) / len(train_sentences))

15

In [21]:
# Setup Text Vectorization Variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 15 # max length the sequences will be (e.g. how many words from a tweet does a model see?)

In [22]:
text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode='int',
                                    output_sequence_length=max_length)

In [23]:
# fit the text vectorizer to the training text
"""
Computes a vocabulary of string terms from tokens in a dataset. Calling adapt() on 
Text Vectorization layer is an alternative to passing in a precomputed vocabulary
on construction via the vocabulary argument. A Text Vectorization layer should 
should always be either adapted over a dataset or supplied with a vocabulary.
"""
text_vectorizer.adapt(train_sentences)

In [24]:
# Sample a sentence and tokenize it
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [25]:
# Choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
vectorize_sentence = text_vectorizer([random_sentence])
print(f"Original Text: \n {random_sentence} \n\n Vectorized Text: \n {vectorize_sentence}")

Original Text: 
 Wow Crackdown 3 uses multiple servers in multiplayer?!?! U can destroy whole buildings?!?! #copped 

 Vectorized Text: 
 [[ 970 3966  118 2626 2851 4659    4 3606  142   71  305  292   95    1
     0]]


In [26]:
# Get Unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary() # get all the unique words from the training data
top_5_words = words_in_vocab[:5] # get the most common words
bottom_5_words = words_in_vocab[-5:] # get the least common words

In [27]:
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"5 most common words: {top_5_words}")
print(f"5 least common words: {bottom_5_words}")

Number of words in vocab: 10000
5 most common words: ['', '[UNK]', 'the', 'a', 'in']
5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


## Creating an Embedding Using the Embedding Layer

Turns positive integers into dense vectors of fixed size

* `input_dim` - the size of the vocabulary
* `output_dim` - the size of the output embedding vector, for example, a value of 100 would mean each token gets represented by a vector 100 long
* `input_length` - length of the sequences being passed to the embedding layer

In [28]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length, # set input shape
                             output_dim=128, # output shape
                             embeddings_initializer='uniform',
                             input_length=max_length # how long is each input
                             )

embedding

In [29]:
# Get a Random sentence from the training set
random_sentence = random.choice(train_sentences)

print(f"Original Text:\n {random_sentence} \n\nEmbedded Version: ")

# Embed the random sentence (turn positive integers into dense vectors of fixed size)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original Text:
 @nycdivorcelaw TRUMP IS A CLIMATE DENIER- algae bloom in the pacific from calif to alska- seeweed in caribean forest fires- SNOWBALL INHOFE 

Embedded Version: 


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.00191194, -0.02992626,  0.02174624, ...,  0.00302727,
         -0.00730379,  0.0346303 ],
        [ 0.04259581, -0.02224317, -0.02177726, ...,  0.0322105 ,
         -0.0227069 ,  0.00523484],
        [-0.04626394, -0.0463458 , -0.02324349, ..., -0.0318409 ,
         -0.01115724,  0.00752554],
        ...,
        [ 0.00794456,  0.00235112,  0.0302866 , ...,  0.02430489,
         -0.02236264,  0.02515768],
        [ 0.0133695 ,  0.0238965 ,  0.04438256, ..., -0.02206298,
          0.01555556, -0.02190062],
        [-0.00191194, -0.02992626,  0.02174624, ...,  0.00302727,
         -0.00730379,  0.0346303 ]]], dtype=float32)>

every single tweet are now in the form of 128 dense vectors. Always using the size as a divisible of 8 will increase the computation speed.

In [30]:
# Checking out single token embedding
random_sentence, sample_embed[0][0], sample_embed[0][0].shape

('@nycdivorcelaw TRUMP IS A CLIMATE DENIER- algae bloom in the pacific from calif to alska- seeweed in caribean forest fires- SNOWBALL INHOFE',
 <tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-1.9119382e-03, -2.9926265e-02,  2.1746244e-02, -1.8007528e-02,
        -4.0272843e-02, -3.4565020e-02, -1.0318302e-02, -4.0053703e-02,
         1.9137982e-02,  3.2780077e-02,  1.3430826e-03, -7.0409887e-03,
         2.5820959e-02, -3.1653523e-02, -4.8954990e-02,  3.4493852e-02,
        -3.1538270e-02,  3.0330662e-02, -2.5676036e-02, -3.3657540e-02,
        -2.7790248e-02,  4.4952396e-02, -1.2484372e-02, -1.6486265e-02,
        -1.2290012e-02,  9.5976703e-03, -4.9855113e-02, -2.1670688e-02,
         4.5809280e-02, -2.2477819e-02, -4.7177188e-03, -3.7623264e-02,
         3.9073478e-02, -8.3962828e-04,  4.2126287e-02, -1.2835443e-02,
         1.5872467e-02,  4.8381519e-02, -3.1540595e-02,  2.0862930e-03,
         5.1646344e-03,  4.6662662e-02, -2.8603151e-04, -3.1220628e-02,
         4.1717

## Modelling a Text Dataset

- Model 0: Naive Bayes - sklearn(baseline)
- Model 1: Feed-Forward Neural Network(dense model)
- Model 2: LSTM Model (RNN)
- Model 3: GRU Model (RNN)
- Model 4: Bidirectional-LSTM Model (RNN)
- Model 5: 1D Conventional Neural Network (CNN)
- Model 6: Tensorflow Hub Pretrained Feature Extractor (using Transfer Learning for NLP)
- Model 7: Same as Model 6 with 10% of training data

## Model 0: Getting a Baseline Model

It's important to create a baseline model to attain a benchmark for future experiments/models to build upon. It's common practice to use non-DL algorithms as a baseline because of their speed and then later using DL to see if it can be improved upon them.

Here the Baseline created is - SkLearn's Multinomial Naive Bayes using the TF-IDF formula to convert the words into numbers

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create Tokenization and Modelling pipeline
model_0 = Pipeline([
    ("tfidf", TfidfVectorizer()), # convert words into numbers using tfidf
    ("clf", MultinomialNB()) # model the text, clf-classification
])

# Fit the Pipeline to Training Data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [32]:
# Evaluate the Baseline Model
baseline_score = model_0.score(test_sentences, test_labels)
print(f"The Baseline Model achieves an accuracy of: {baseline_score*100:.2f}%")

The Baseline Model achieves an accuracy of: 79.27%


In [33]:
# make predictions
baseline_preds = model_0.predict(test_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

In [34]:
# Function to evaluate: accuracy, precision, recall, f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculate model accuracy, precision, recall and f1 score of a binary
  classification model.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred)*100

  # Calculate model precision, recall and f1-score using weighted average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")

  model_results={"accuracy": model_accuracy,
                 "precision": model_precision,
                 "recall": model_recall,
                 "f1": model_f1}

  return model_results

In [35]:
# Get Baseline results
baseline_results = calculate_results(y_true=test_labels,
                                     y_pred=baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

## Model 1: A Simple Dense Model

In [37]:
# Create a tensorboard callback (need to create a new one for each model)
from helper_functions import create_tensorboard_callback

# Creating a directory to save tensorboard logs
SAVE_DIR = "model_logs"

In [40]:
# Build Model with a Functional API
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype=tf.string) # inputs are 1-dimensional strings
x = text_vectorizer(inputs) # turn the input text into numbers
x = embedding(x) # create a embedding of the numberized inputs
outputs = layers.Dense(1, activation="sigmoid")(x) # Create the output layer, want binary outputs so using sigmoid activation function
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense")

In [41]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 dense (Dense)               (None, 15, 1)             129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
Non-trainable params: 0
_________________________________________________________________


In [42]:
# Compile the model
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [ ]:
# fit the model
model_1_history = model_1.fit()